In [9]:
#Create an options chain 

In [10]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm
import datetime

In [11]:
optdta = pd.read_csv("/users/nick/desktop/GithubProjects/creating-a-historical-theoretical-eth-options-chain/dataforoptchain.csv")


In [12]:
optdta.rename(columns={'Unnamed: 0':'DATE'},inplace =True)
optdta.set_index('DATE', drop=True, inplace=True)

In [13]:
#finding the strike prices for each chain
#max weekly strike price is .02*20 = 40% out of the money
#max monthly strike price is 60% out of the money
#max yearly strike price is 150% out of the money

def findstrikes(spot, epoch):
    strikes = []
    strikes.append(spot)
    if(epoch == 'w'):
        for i in range (1,21):
            strikes.append(np.around(spot - spot*.02*i))
            strikes.append(np.around(spot + spot*.02*i))
    
    elif epoch == 'm':
        for i in range (1,21):
            strikes.append(np.around((spot - spot*.03*i)/5)*5)
            strikes.append(np.around((spot + spot*.03*i)/5)*5)
    
    elif epoch == 'y':
        for i in range(1,21):
            strikes.append(np.around((spot - spot*.06*i)/10)*10)
            strikes.append(np.around((spot + spot*.06*i)/10)*10)
            
    strikes.sort()
    return strikes

In [14]:
#black scholes options premium prediction

#S = spot price of asset
#K = strike price of option
#T = time in years
#r = risk free interest rate
#sigma = annualized vol (vix as a percentage)

N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

In [15]:
def VolAdjuster(vol):
    vols = []
    #manually reverse the range
    z = 20
    for i in range (1,21):
        vols.append(vol + vol*.005*z)
        z-=1
    vols.append(vol)
    for i in range(1,21):
        vols.append(vol + vol*.005*i)
    
#balance out the total implied volatility because VIX uses many different strikes

    vols = [z - .05*z for z in vols]
    return vols

In [16]:
#calls

chain = pd.DataFrame(columns = ['Strike'])
i = 3

chain['Strike'] = findstrikes(optdta.iloc[i]['Value'], 'w')


#make a naive vol smile, lets say 10% on each side at max OTM-ness

chain['AdjVix'] = VolAdjuster(optdta.iloc[i]['VIX'])
chain['StDate'] = optdta.index[i]
chain['ExpDate'] = optdta.index[i+7]
chain['StSpot'] = optdta.iloc[i]['Value']
chain['ExpSpot'] = optdta.iloc[i+7]['Value']
chain['StOptPrice'] = BS_CALL(chain['StSpot'], chain['Strike'],(7/365),optdta.iloc[i]['DGS10'], chain['AdjVix'])
chain['EndOptPrice'] = chain.iloc[i+7]['ExpSpot'] - chain['Strike']
chain['EndOptPrice'] = chain['EndOptPrice'].clip(lower = 0)

chain['EthChg'] = chain['ExpSpot'] - chain['StSpot']
chain['IntValue'] = chain['StSpot'] - chain['Strike']
chain['IntValue'] = chain['IntValue'].clip(lower = 0)
chain['BuyerProfit'] = chain['EndOptPrice'] - chain['StOptPrice']
chain['ExtPrem'] = chain['StOptPrice'] - chain['IntValue']
chain['EthProfit'] = chain['EthChg'] - chain['StSpot'] + chain['ExpSpot']
chain['truelpprofit'] = np.nan
chain['truelpprofit'] = np.where((chain['IntValue'] == 0) & (chain['ExpSpot'] < chain['Strike']),chain['EthChg'] + chain['StOptPrice'],chain['truelpprofit'])
chain['truelpprofit'] = np.where((chain['IntValue'] > 0) & (chain['ExpSpot'] > chain['Strike']), chain['ExtPrem'],chain['truelpprofit'])
chain['truelpprofit'] = np.where((chain['IntValue'] > 0) & (chain['ExpSpot'] < chain['Strike']), chain['StOptPrice']-chain['Strike']+chain['ExpSpot'],chain['truelpprofit'])
chain['truelpprofit'] = np.where((chain['IntValue'] == 0) & (chain['ExpSpot'] > chain['Strike']), chain['StOptPrice'] + chain['Strike']-chain['StSpot'],chain['truelpprofit'])


chain


,Strike,AdjVix,StDate,ExpDate,StSpot,ExpSpot,StOptPrice,EndOptPrice,EthChg,IntValue,BuyerProfit,ExtPrem,EthProfit,truelpprofit
0,113.00,1.043642,2020-04-18,2020-04-25,187.81,194.39,76.210698,81.39,6.58,74.81,5.179302,1.400698,13.16,1.400698
1,116.00,1.038898,2020-04-18,2020-04-25,187.81,194.39,73.248639,78.39,6.58,71.81,5.141361,1.438639,13.16,1.438639
2,120.00,1.034154,2020-04-18,2020-04-25,187.81,194.39,69.300343,74.39,6.58,67.81,5.089657,1.490343,13.16,1.490343
3,124.00,1.029410,2020-04-18,2020-04-25,187.81,194.39,65.354415,70.39,6.58,63.81,5.035585,1.544415,13.16,1.544415
4,128.00,1.024666,2020-04-18,2020-04-25,187.81,194.39,61.412899,66.39,6.58,59.81,4.977101,1.602899,13.16,1.602899
5,131.00,1.019922,2020-04-18,2020-04-25,187.81,194.39,58.461294,63.39,6.58,56.81,4.928706,1.651294,13.16,1.651294
6,135.00,1.015179,2020-04-18,2020-04-25,187.81,194.39,54.536411,59.39,6.58,52.81,4.853589,1.726411,13.16,1.726411
7,139.00,1.010435,2020-04-18,2020-04-25,187.81,194.39,50.629759,55.39,6.58,48.81,4.760241,1.819759,13.16,1.819759
8,143.00,1.005691,2020-04-18,2020-04-25,187.81,194.39,46.750992,51.39,6.58,44.81,4.639008,1.940992,13.16,1.940992
9,146.00,1.000947,2020-04-18,2020-04-25,187.81,194.39,43.866090,48.39,6.58,41.81,4.523910,2.056090,13.16,2.056090
